# AutoGen Core: Runtime Architecture & Agent Communication

This notebook introduces **AutoGen Core**, a low-level framework for building distributed agent systems. Unlike AutoGen AgentChat (which provides pre-built agent abstractions), AutoGen Core offers fine-grained control over:
1. **Message routing**: Custom message types and handlers
2. **Runtime management**: Standalone vs. distributed execution
3. **Agent lifecycle**: Registration, initialization, and shutdown

**Key Pattern:** AutoGen Core decouples agent logic from message delivery—agents define behavior, the runtime handles communication.

In [ ]:
# Import dependencies
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)

## Phase 1: Define Custom Message Type

In [ ]:
# Custom Message Schema
@dataclass
class Message:
    content: str

## Phase 2: Define Routed Agent

**Agent ID Components:**
- `agent.id.type`: Agent category (e.g., "SimpleAgent")
- `agent.id.key`: Unique instance identifier (e.g., "default")

In [ ]:
# Simple Agent Implementation
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("SimpleAgent")

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"{self.id.type}-{self.id.key}: I disagree with '{message.content}'")

## Phase 3: Initialize Runtime & Register Agent

In [ ]:
# Create Runtime
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

In [ ]:
# Start Runtime & Send Message
runtime.start()

agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(f">>> {response.content}")

# Cleanup
await runtime.stop()
await runtime.close()

## Phase 4: Integrate AutoGen AgentChat Agents

In [ ]:
# Wrapper for AgentChat Assistant
class LLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)

In [ ]:
# Register Multiple Agents
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await LLMAgent.register(runtime, "LLMAgent", lambda: LLMAgent())

runtime.start()

# Multi-Agent Conversation
response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
print(f">>> {response.content}")

response = await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(f">>> {response.content}")

response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))
print(f">>> {response.content}")

await runtime.stop()
await runtime.close()

## Phase 5: Multi-Agent Game (Rock, Paper, Scissors)

In [ ]:
# Player Agents (OpenAI vs. Ollama)
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OllamaChatCompletionClient(model="llama3.2", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [ ]:
# Game Orchestrator Agent
JUDGE = "You are judging rock, paper, scissors. The players chose:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = "Play rock, paper, scissors. Respond with only one word: rock, paper, or scissors."
        msg = Message(content=instruction)
        
        # Send to both players
        response1 = await self.send_message(msg, AgentId("player1", "default"))
        response2 = await self.send_message(msg, AgentId("player2", "default"))
        
        # Judge winner
        result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
        judgement = f"{JUDGE}{result}Who wins?"
        text_msg = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([text_msg], ctx.cancellation_token)
        return Message(content=result + response.chat_message.content)

In [ ]:
# Register & Execute Game
runtime = SingleThreadedAgentRuntime()
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))

runtime.start()

# Play Game
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

await runtime.stop()
await runtime.close()